In [1]:
import pandas as pd
import sys
import os
import time
import random

In [2]:
# Import libraries
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Time imports
import random
import datetime
import time

# Other imports
import re as re
import pandas as pd
import os
import sys
import numpy as np

import pymysql
import sqlalchemy as alch
from getpass import getpass

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
mercadona_user = os.getenv("mercadona_user")
mercadona_password = os.getenv("mercadona_password")

In [3]:
def get_purchase_history(zip, mercadona_user, mercadona_password, headless=True):
    """
    """
    # create a ChromeOptions object
    options = Options()

    # add the headless argument if passed
    if headless:
        options.add_argument('--headless')

    # specify the path to your web driver
    driver = webdriver.Chrome(options=options)

    # navigate to the login page
    driver.get('https://www.mercadona.es/')

    # enter the postal code and submit it
    postal_code = driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Código postal"]').send_keys(zip)
    submit_button = driver.find_element(By.CSS_SELECTOR, 'input.postal-code-form__button').click()

    # Accept cookies
    accept_button = driver.find_element(By.XPATH, "//button[contains(text(),'Aceptar todas')]").click()

   # wait for the page to load
    driver.implicitly_wait(10)

    """
    # click the "Identifícate" link
    identificate_link = driver.find_element(By.CSS_SELECTOR, 'div.account__title')
    identificate_link.click()
    """

    # click the "Identifícate" button
    identificate_button = driver.find_element(By.CSS_SELECTOR, 'a[href*="authenticate-user"]')
    identificate_button.click()

    # enter the email address
    email = driver.find_element(By.CSS_SELECTOR, 'input[name="email"]')
    email.send_keys(mercadona_user)

    # wait random time
    #sleep(random.uniform(1.5, 1.9))

    # click the "Siguiente" button
    next_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    next_button.click()

    # wait random time
    #sleep(random.uniform(1.5, 1.9))

    # enter the password
    password = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
    password.send_keys(mercadona_password)

    # click the "Entrar" button
    entrar_button = driver.find_element(By.CSS_SELECTOR, 'button[data-test="do-login"]')
    entrar_button.click()

    # drop down de "Hola Andrés"
    personal_section = driver.find_element(By.CLASS_NAME, "drop-down__trigger")
    personal_section.click()

    # abrir "Mis pedidos"
    pedidos = driver.find_element(By.CLASS_NAME, "subhead1-r.account-menu__item")
    pedidos.click()

    """
    # Initialize list of dictionaries (to be turned into a dataframe) and product counter for feedback
    list_of_dicts = []
    product_count = 0

    # Iterate over the "product-cell" elements (products)
    for i in range(len(product_cells)):
        # Scroll to the current product cell and wait for the "product-cell" element to be clickable
        driver.execute_script("arguments[0].scrollIntoView();", product_cells[i])
        product_cell = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[data-test='product-cell']")))

        # wait for the description element to be present
        descripcion = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.private-product-detail__description')))

        # Give feedback to user by printing the current product
        print(f'\rScraping "{i+1}: {product_cells[i].text[0:15]}..." product...                                                                              ', end='')
        sys.stdout.flush()

        # Initialize the dictionary that will be apended to the list (that will later be our DataFrame)
        info_prod={}

        # Scrapes data if available, when it is not it saves "Not available"
        # Product name
        try:
            info_prod["product"] = driver.find_element(By.CSS_SELECTOR, 'h1.title2-r.private-product-detail__description').text
        except:
            info_prod["product"] = "Not available"
        # Product_type
        try:
            info_prod["product_type"] = driver.find_element(By.CSS_SELECTOR, 'span.headline1-r:nth-child(1)').text
        except:
            info_prod["product_type"] = "Not available"
        # Product_volume
        try:
            info_prod["product_volume"] = driver.find_element(By.CSS_SELECTOR, 'span.headline1-r:nth-child(2)').text
        except:
            info_prod["product_volume"] = "Not available"
        # Price per unit (€ / L)
        try:
            info_prod["product_price_per_unit"] = driver.find_element(By.CSS_SELECTOR, 'span.headline1-r:nth-child(3)').text
        except:
            info_prod["product_price_per_unit"] = "Not available"
        # Product_price
        try:
            info_prod["product_price"] = float(driver.find_element(By.CSS_SELECTOR, 'p.product-price__unit-price.large-b').text.replace("€","").strip().replace(",","."))
        except:
            info_prod["product_price"] = "Not Available"
        # Product unit (e.g.: L)
        try:
            info_prod["product_unit"] = driver.find_element(By.CSS_SELECTOR, 'p.product-price__extra-price.title1-r').text
        except:
            info_prod["product_unit"] = "Not Available"
        # Product category
        try:
            info_prod["product_category"] = driver.find_element(By.CSS_SELECTOR, 'span.subhead1-r').text
        except:
            info_prod["product_category"] = "Not Available"
        # Product Subcategory
        try:
            info_prod["product_subcategory"] = driver.find_element(By.CSS_SELECTOR, 'span.subhead1-sb').text
        except:
            info_prod["product_subcategory"] = "Not Available"
        # Url, Product code (from URL), and scraped time
        info_prod["product_url"] = driver.current_url
        info_prod["product_code"] = driver.current_url.split("/")[4]
        info_prod["collected_timestamp"] = datetime.datetime.now()

        # Appends the current row (product) to the list of dicts (will be turned to a DataFrame) and advances the counter
        list_of_dicts.append(info_prod)
        product_count += 1
        
        # Send the 'esc' key and the back command to exit the product info page. Does it until we are moved back to the product grid (URL contains "categories")
        while "categories" not in driver.current_url:
            driver.back()
            driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.ESCAPE)

        # Click on the next "product-cell" element, if available
        if i < len(product_cells) - 1:
            next_product_cell = product_cells[i+1]
            next_product_cell.click()

            # Wait for the page to load
            WebDriverWait(driver, 10).until(EC.url_changes(current_url))

            # If we get an error for too many requests, exit the function by returning "Error" and closing the browser window.
            try:
                len(driver.find_element(By.XPATH, '//button[contains(text(), "Entendido")]').text) > 0
                driver.quit()
                return "error"

            except:
                continue
        
    # Creates the Data Frame to return from the list of dictionaries created and closes the browser window
    ret_df = pd.DataFrame(list_of_dicts)
    driver.quit()
    """
    return 